# Preparation Script

### Setup

In [1]:
!pip install -r requirements.txt

In [ ]:
!pip install wiki_dump_parser
!pip install pyarrow
!apt install pigz

In [11]:
import wiki_dump_parser
import sys

from os import listdir, remove, system, devnull
from os.path import join
import urllib.request
import pathlib
import subprocess
import pandas as pd


# Disable
def blockPrint():
    sys.stdout = open(devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__

dataFolder="./data/"
backupDate="20200301"
language = 'uk' # We can also loop through different languages as next step.

### Download Files
First the files will be downloaded from given target.

Then we need to convert xml dumps to csvs for further processing. The files should be residing in ```data/raw/xml```. Don't forget putting xml's under their respective folder, ex: ```data/raw/xml/en/enwiki-20200401-stub-meta-history1.xml```.

In [12]:
def convertToParquet(csvFileName):
    # Convert the csv to parquet for faster processing.
    df = pd.read_csv(csvFileName, quotechar ='|')
    df.page_title = df.page_title.astype('string') 
    parquetFileName = csvFileName.replace(".csv", ".parquet")
    df.to_parquet(parquetFileName, engine='pyarrow')
    remove(csvFileName)
    print(f"Completed Processing {xmlDataFile}!")

def processXmlData(xmlDataFile):
    print(f"Processing {xmlDataFile}")
    blockPrint()
    wiki_dump_parser.xml_to_csv(xmlDataFile)
    enablePrint()
    csvFileName = xmlDataFile.replace(".xml",".csv")
    remove(xmlDataFile)

def processZip(zipPath):
    print(f"Extracting: {zipPath}")
    result = subprocess.run(['./unpigz.exe', '-d', zipPath])
    extractedFileName = zipPath.replace(".gz","")
    print(f"Extracted file: {zipPath}")
    processXmlData(extractedFileName)


packs = [1] #list(map(lambda x: x+1, range(27)))

for currentPack in packs:
    languageFolder = f"{dataFolder}{language}"
    pathlib.Path(languageFolder).mkdir(parents=True, exist_ok=True)
    fileName = f"{language}wiki-{backupDate}-stub-meta-history.xml.gz"
    backupUrl = f"http://ftp.acc.umu.se/mirror/wikimedia.org/dumps/{language}wiki/{backupDate}/{fileName}"
    print(backupUrl)

    filePath = f"{languageFolder}/{fileName}"
    urllib.request.urlretrieve(backupUrl, filePath)
    
    processZip(filePath)
    break
    
    

UnicodeEncodeError: 'charmap' codec can't encode characters in position 10-14: character maps to <undefined>